In [11]:
json_file = "/vc_data_blob/users/kevihuang/data/prm800k/prm800k/data/phase2_train.jsonl" # 100K. Generally problems where PRM scored high, but actually wrong solution (tricky)

# "/vc_data_blob/users/kevihuang/data/prm800k/prm800k/data/phase2_test.jsonl" # 3K
# "/vc_data_blob/users/kevihuang/data/prm800k/prm800k/data/phase2_train.jsonl" # 100K

import jsonlines
data = []
with jsonlines.open(json_file) as reader:
    for obj in reader:
        data.append(obj)
print(len(data))

97782


In [12]:
data[0]

{'labeler': '340d89bc-f5b7-45e9-b272-909ba68ee363',
 'timestamp': '2023-03-19T02:54:52.529607',
 'generation': 9,
 'is_quality_control_question': False,
 'is_initial_screening_question': False,
 'question': {'problem': 'The first four terms in an arithmetic sequence are $x+y$, $x-y$, $xy$, and $x/y$, in that order. What is the fifth term? Express your answer as a common fraction.',
  'ground_truth_solution': 'Since the difference of the first two terms is $-2y$, the third and fourth terms of the sequence must be $x-3y$ and $x-5y$. Thus \\[\nx-3y = xy \\quad\\text{and}\\quad x-5y = \\frac{x}{y},\n\\]so $xy - 5y^{2} = x.$ Combining these equations we obtain \\[\n(x - 3y) - 5y^{2}= x\\quad\\text{and, therefore, }\\quad -3y - 5y^{2} = 0.\n\\]Since $y$ cannot be 0, we have $y = -\\frac{3}{5}$, and it follows that $x = -\\frac{9}{8}$. The fifth term in the sequence is $x - 7y\n= \\boxed{\\frac{123}{40}}$.',
  'ground_truth_answer': '\\frac{123}{40}',
  'pre_generated_steps': ['To find the fi

In [13]:
data[0]['question']

{'problem': 'The first four terms in an arithmetic sequence are $x+y$, $x-y$, $xy$, and $x/y$, in that order. What is the fifth term? Express your answer as a common fraction.',
 'ground_truth_solution': 'Since the difference of the first two terms is $-2y$, the third and fourth terms of the sequence must be $x-3y$ and $x-5y$. Thus \\[\nx-3y = xy \\quad\\text{and}\\quad x-5y = \\frac{x}{y},\n\\]so $xy - 5y^{2} = x.$ Combining these equations we obtain \\[\n(x - 3y) - 5y^{2}= x\\quad\\text{and, therefore, }\\quad -3y - 5y^{2} = 0.\n\\]Since $y$ cannot be 0, we have $y = -\\frac{3}{5}$, and it follows that $x = -\\frac{9}{8}$. The fifth term in the sequence is $x - 7y\n= \\boxed{\\frac{123}{40}}$.',
 'ground_truth_answer': '\\frac{123}{40}',
 'pre_generated_steps': ['To find the fifth term, I need to identify the common difference of the arithmetic sequence and add it to the fourth term.',
  'The common difference is the same for any consecutive pair of terms, so I can use any of them to

In [14]:
# Add an iscorrect field 
for entry in data:
    question = entry.get('question', {})
    question['iscorrect'] = (question.get('pre_generated_answer') ==
                             question.get('ground_truth_answer'))

In [15]:
data[0]

{'labeler': '340d89bc-f5b7-45e9-b272-909ba68ee363',
 'timestamp': '2023-03-19T02:54:52.529607',
 'generation': 9,
 'is_quality_control_question': False,
 'is_initial_screening_question': False,
 'question': {'problem': 'The first four terms in an arithmetic sequence are $x+y$, $x-y$, $xy$, and $x/y$, in that order. What is the fifth term? Express your answer as a common fraction.',
  'ground_truth_solution': 'Since the difference of the first two terms is $-2y$, the third and fourth terms of the sequence must be $x-3y$ and $x-5y$. Thus \\[\nx-3y = xy \\quad\\text{and}\\quad x-5y = \\frac{x}{y},\n\\]so $xy - 5y^{2} = x.$ Combining these equations we obtain \\[\n(x - 3y) - 5y^{2}= x\\quad\\text{and, therefore, }\\quad -3y - 5y^{2} = 0.\n\\]Since $y$ cannot be 0, we have $y = -\\frac{3}{5}$, and it follows that $x = -\\frac{9}{8}$. The fifth term in the sequence is $x - 7y\n= \\boxed{\\frac{123}{40}}$.',
  'ground_truth_answer': '\\frac{123}{40}',
  'pre_generated_steps': ['To find the fi

In [16]:
# Assuming data is already loaded and each entry's question dictionary has an 'iscorrect' key.

total = len(data)
if total == 0:
    print("No entries found in data.")
else:
    true_count = sum(1 for entry in data if entry['label'].get('finish_reason', {}) == "solution")
    false_count = total - true_count

    percentage_true = (true_count / total) * 100
    percentage_false = (false_count / total) * 100

    print(f"Percentage iscorrect = True: {percentage_true:.2f}%")
    print(f"Percentage iscorrect = False: {percentage_false:.2f}%")


Percentage iscorrect = True: 11.93%
Percentage iscorrect = False: 88.07%


In [17]:
# Create eval data

for entry in data:
    # Extract the question text (here we assume it's stored under the 'problem' key)
    question_text = entry.get('question', {}).get('problem', '')
    
    # Concatenate pre_generated_steps into a single string with newline separation.
    steps = entry.get('question', {}).get('pre_generated_steps', [])
    response_text = "\n".join(steps)
    
    # Determine the correctness based on finish_reason.
    question = entry.get('question', {})
    if question.get('pre_generated_answer') == question.get('ground_truth_answer') and entry['label'].get('finish_reason', {}) == "solution":
        is_correct = True # Only add solutions that are obviously correct thorugh an exact match
    elif question.get('pre_generated_answer') != question.get('ground_truth_answer') and entry['label'].get('finish_reason', {}) == "found_error":
        is_correct = False # Only add solutions that are obviously incorrect from a labeler concluding an error was found AND there wasn't an exact match (did not conlcude to the correct answer, nor was a bad problem or give up): https://github.com/openai/prm800k. High chance that the solution was wrong
    else:
        is_correct = None # otherwise it might be ambiguous
    
    # Add the new 'rm_eval' key with the desired structure.
    entry['rm_eval'] = {
        'question': question_text,
        'response': response_text,
        'iscorrect': is_correct,
        'gt_answer': entry.get('question', {}).get('ground_truth_answer', '')
    }

    # Add the veriy prompt and final answer
    rf_prompt = f'''You are a math teacher. Grade the Solution, verifying correctness step by step.

Question: "{question_text.strip()}"

Solution: "{response_text.strip()}"

Please output your Solution verification in the following format:
<think> Your Solution verification here </think>
<answer> Yes or No </answer>

Math Teacher Response: <think>\n'''
    entry["prompt"] = [{
        "from": "user",
        "value": rf_prompt
    }]
    entry["final_answer"] = "yes" if is_correct else "no"


In [18]:
data[0]['rm_eval']

{'question': 'The first four terms in an arithmetic sequence are $x+y$, $x-y$, $xy$, and $x/y$, in that order. What is the fifth term? Express your answer as a common fraction.',
 'response': 'To find the fifth term, I need to identify the common difference of the arithmetic sequence and add it to the fourth term.\nThe common difference is the same for any consecutive pair of terms, so I can use any of them to find it.\nFor example, using the first and second terms, I can write $x-y = x+y + d$, where $d$ is the common difference.\nSolving for $d$, I get $d = -2y$.\nUsing another pair of terms, such as the second and third, I can check if this value of $d$ is consistent.\nI have $xy = x-y + d$, so substituting $d = -2y$, I get $xy = x-y - 2y$.\nSimplifying, I get $xy = x - 3y$.\nThis seems like a reasonable equation, so I will assume that $d = -2y$ is correct.\nNow, to find the fifth term, I need to add $d$ to the fourth term.\nThe fourth term is $x/y$, so the fifth term is $x/y + d = x

In [19]:
import json
import random

# Filter entries where rm_eval.iscorrect is True and False.
correct_entries = [entry for entry in data if entry.get('rm_eval', {}).get('iscorrect') is True and entry['question'].get('ground_truth_answer', '') != ''] # high chance actually correct because final answers actually matched
incorrect_entries = [entry for entry in data if entry.get('rm_eval', {}).get('iscorrect') is False and entry['question'].get('ground_truth_answer', '') != ''] # high chance actually incorrect because labeler found error and final answers did not match

# Check if there are at least 100 entries in each category.
if len(correct_entries) < 100 or len(incorrect_entries) < 100:
    print("Not enough data to sample 100 correct and 100 incorrect entries.")
else:
    # Randomly sample 100 from each.
    sample_correct = random.sample(correct_entries, 100)
    sample_incorrect = random.sample(incorrect_entries, 100)
    
    # Combine and shuffle the samples.
    combined_sample = sample_correct + sample_incorrect
    random.shuffle(combined_sample)
    
    # Define the output file path.
    output_file = '/data/users/kevihuang/projects/Open-Reasoner-Zero/data/eval_data/prm800k_100_correct_100_incorrect_rm_eval.json'
    
    # Save the combined_sample to the file in JSON format.
    with open(output_file, 'w') as f:
        json.dump(combined_sample, f, indent=2)
    
    print(f"Saved {len(combined_sample)} entries to {output_file}")


Saved 200 entries to /data/users/kevihuang/projects/Open-Reasoner-Zero/data/eval_data/prm800k_100_correct_100_incorrect_rm_eval.json
